In [1]:
# Core libraries and paths.

from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

DIR = Path("ml-100k")  
f_data = DIR / "u.data"
f_item = DIR / "u.item"
f_user = DIR / "u.user"

## Load Data
Read MovieLens files.

In [2]:
# ratings (tab-separated)
ratings = pd.read_csv(
    f_data, sep="\t", header=None,
    names=["user_id", "movie_id", "rating", "timestamp"],
    dtype={"user_id":"int32","movie_id":"int32","rating":"int8","timestamp":"int64"}
)

# movies (pipe-separated, latin-1)
movie_cols = [
    "movie_id","title","release_date","video_release_date","imdb_url",
    "unknown","Action","Adventure","Animation","Children's","Comedy","Crime",
    "Documentary","Drama","Fantasy","Film-Noir","Horror","Musical","Mystery",
    "Romance","Sci-Fi","Thriller","War","Western"
]
movies = pd.read_csv(f_item, sep="|", header=None, names=movie_cols, encoding="latin-1")

# users (pipe-separated)
users = pd.read_csv(
    f_user, sep="|", header=None,
    names=["user_id","age","gender","occupation","zip_code"],
    dtype={"user_id":"int32","age":"int16","gender":"category","occupation":"category","zip_code":"string"}
)

## Timestamp Conversion
Transform UNIX timestamps to `datetime`.

In [3]:
ratings["datetime"] = pd.to_datetime(ratings["timestamp"], unit="s")
ratings.drop(columns="timestamp", inplace=True)

## Sanity Checks
Shapes, unique IDs, nulls.

In [4]:
summary = pd.DataFrame(
    [
        ["Dataset Shape (Ratings)", str(ratings.shape)],
        ["Dataset Shape (Movies)",  str(movies.shape)],
        ["Dataset Shape (Users)",   str(users.shape)],
        ["Unique Users in Ratings", ratings["user_id"].nunique()],
        ["Unique Movies in Ratings",ratings["movie_id"].nunique()],
        ["Missing Values in Ratings", int(ratings.isna().sum().sum())],
        ["Missing Movie Titles",       int(movies["title"].isna().sum())],
        ["Missing Values in Users",    int(users.isna().sum().sum())],
    ],
    columns=["Metric","Value"]
)
display(summary.style.hide(axis="index"))

Metric,Value
Dataset Shape (Ratings),"(100000, 4)"
Dataset Shape (Movies),"(1682, 24)"
Dataset Shape (Users),"(943, 5)"
Unique Users in Ratings,943
Unique Movies in Ratings,1682
Missing Values in Ratings,0
Missing Movie Titles,0
Missing Values in Users,0


## Data Preview  
Quick peek at `ratings`, `movies`, and `users` tables.  

In [5]:
display(
    ratings.head(3).style.set_caption("Ratings Sample"),
    movies[["movie_id","title","release_date"]].head(3).style.set_caption("Movies Sample"),
    users.head(3).style.set_caption("Users Sample")
)

,user_id,movie_id,rating,datetime
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36


,movie_id,title,release_date
0,1,Toy Story (1995),01-Jan-1995
1,2,GoldenEye (1995),01-Jan-1995
2,3,Four Rooms (1995),01-Jan-1995


,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067


## Missingness Overview
Summarize total null values across datasets to identify where cleaning is required.


In [6]:
nulls_summary = pd.DataFrame({
    "Dataset": ["Ratings", "Movies", "Users"],
    "Total Nulls": [
        ratings.isnull().sum().sum(),
        movies.isnull().sum().sum(),
        users.isnull().sum().sum(),
    ]
})

display(nulls_summary.style.hide(axis="index"))

Dataset,Total Nulls
Ratings,0
Movies,1686
Users,0


In [7]:
# Nulls by column in Movies
movies_nulls = movies.isnull().sum()
movies_nulls = movies_nulls[movies_nulls > 0].to_frame("Missing Values")

display(movies_nulls)

,Missing Values
release_date,1
video_release_date,1682
imdb_url,3


## Data Cleaning (Movies)
Remove the all-null column `video_release_date` and drop residual rows with critical missing fields.


In [8]:
# Drop column with all nulls
movies = movies.drop(columns=["video_release_date"])

# Drop rows with any remaining nulls
movies = movies.dropna()

print("> Cleaned Movies shape:", movies.shape)

> Cleaned Movies shape: (1679, 23)


## Ratings × Movies Join
Attach titles (and release dates) to each rating to produce a unified interaction table.


In [9]:
ratings_movies = ratings.merge(movies[["movie_id", "title", "release_date"]], on="movie_id", how="left")

print(f"Merged shape: {ratings_movies.shape}")
ratings_movies.head(5)

Merged shape: (100000, 6)


,user_id,movie_id,rating,datetime,title,release_date
0,196,242,3,1997-12-04 15:55:49,Kolya (1996),24-Jan-1997
1,186,302,3,1998-04-04 19:22:22,L.A. Confidential (1997),01-Jan-1997
2,22,377,1,1997-11-07 07:18:36,Heavyweights (1994),01-Jan-1994
3,244,51,2,1997-11-27 05:02:03,Legends of the Fall (1994),01-Jan-1994
4,166,346,1,1998-02-02 05:33:16,Jackie Brown (1997),01-Jan-1997


In [10]:
# Check nulls after merge
print("Nulls after merge:")
print(ratings_movies.isnull().sum())

Nulls after merge:
user_id          0
movie_id         0
rating           0
datetime         0
title           13
release_date    13
dtype: int64


In [11]:
ratings_movies = ratings_movies.dropna(subset=["title", "release_date"])

## Exports
Persist clean tables for reuse in later notebooks: `ratings.csv`, `movies.csv`, `users.csv`, and the merged `ratings_movies.csv`.


In [12]:
out = Path("Datasets")
out.mkdir(exist_ok=True)

ratings.to_csv(out / "ratings.csv", index=False, encoding="utf-8")
movies.to_csv(out / "movies.csv", index=False, encoding="utf-8")
users.to_csv(out / "users.csv", index=False, encoding="utf-8")
ratings_movies.to_csv(out / "ratings_movies.csv", index=False, encoding="utf-8")

print("Saved to 'Datasets':")
for f in ["ratings.csv","movies.csv","users.csv","ratings_movies.csv"]:
    print(" --> ", f)

Saved to 'Datasets':
 -->  ratings.csv
 -->  movies.csv
 -->  users.csv
 -->  ratings_movies.csv
